# Dilated Temporal Convolutional Network

Implementation of a dilated TCN loosely based on the dilated temporal convolutional network found in this paper: https://arxiv.org/abs/1611.05267

Model is not an exact replica of the one described in the paper, and a flattening layer was added before the output layer. Hyperparameters are also slightly different regarding the filter in each convolutional layer, and the last two layers feeding into the output layer are temporal convolutional layers without dilations.

This architecture took significantly less time to train than the encoder-decoder network, while obtaining a high training and testing accura




Tested the model on sequential MNIST data, which refers to a flattened version of each MNIST image. 

Model obtained a 99.68% training accuracy and 97.02% testing accuracy.

Training time was around 10 and a half minutes on an Nvidia GTX 1060

In [3]:
import tensorflow as tf
import keras.backend as K
from keras import optimizers
from keras.layers import Convolution1D, Dense, MaxPool1D
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.layers import Activation, Dense

In [17]:
from keras.datasets import mnist
batch_size = 128
num_classes = 10
epochs = 15

img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Reshaping it so that it is (60000, 784, 1) instead of (60000, 28, 28, 1) since it's not channels first
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows * img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows * img_cols)
    input_shape = (1, img_rows * img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols, 1)
input_shape = (img_rows * img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


print('Input shape: %s \nX_train shape: %s \nY_train shape: %s' % (input_shape, x_train.shape, y_train.shape))

Input shape: (784, 1) 
X_train shape: (60000, 784, 1) 
Y_train shape: (60000, 10)


In [24]:


input_layer = Input(shape=(input_shape ))

#Series of temporal convolutional layers with dilations increasing by powers of 2.
conv_1 = keras.layers.convolutional.Conv1D(filters=128,
                                                   kernel_size=2,
                                                   dilation_rate=1,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(input_layer)

conv_2 = keras.layers.convolutional.Conv1D(filters=128,
                                                   kernel_size=2,
                                                   dilation_rate=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(conv_1)

conv_3 = keras.layers.convolutional.Conv1D(filters=128,
                                                   kernel_size=2,
                                                   dilation_rate=4,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(conv_2)

conv_4 = keras.layers.convolutional.Conv1D(filters=128,
                                                   kernel_size=2,
                                                   dilation_rate=8,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(conv_3)

#Two relu convolutional layers without dilations to feed into the output layer
conv_5 = keras.layers.convolutional.Conv1D(filters=128,
                                                   kernel_size=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(conv_4)

conv_6 = keras.layers.convolutional.Conv1D(filters=128,
                                                   kernel_size=2,
                                                   padding='causal',
                                                   strides=1,
                                                   input_shape=input_shape,
                                                   activation = 'relu')(conv_5)
#Flatten to match dimensions of our y_test
flat = keras.layers.Flatten()(conv_5)

output_layer = Dense(num_classes, activation='softmax')(flat)

TCN = Model(inputs=input_layer, outputs=output_layer)

In [25]:
TCN.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="model_seqMnistDilated.h5",
                               verbose=0,
                               save_best_only=True)

TCN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 784, 1)            0         
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 784, 128)          384       
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 784, 128)          32896     
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 784, 128)          32896     
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 784, 128)          32896     
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 784, 128)          32896     
_________________________________________________________________
flatten_4 (Flatten)          (None, 100352)            0         
__________

In [26]:
TCN.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks = [checkpointer])


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 44s - loss: 0.2028 - acc: 0.9385 - val_loss: 0.1034 - val_acc: 0.9677
Epoch 2/15
60000/60000 [==============================] - 42s - loss: 0.0994 - acc: 0.9696 - val_loss: 0.0970 - val_acc: 0.9714
Epoch 3/15
60000/60000 [==============================] - 42s - loss: 0.0733 - acc: 0.9773 - val_loss: 0.0941 - val_acc: 0.9711
Epoch 4/15
60000/60000 [==============================] - 42s - loss: 0.0544 - acc: 0.9827 - val_loss: 0.1052 - val_acc: 0.9694
Epoch 5/15
60000/60000 [==============================] - 42s - loss: 0.0362 - acc: 0.9887 - val_loss: 0.1056 - val_acc: 0.9728
Epoch 6/15
60000/60000 [==============================] - 42s - loss: 0.0252 - acc: 0.9915 - val_loss: 0.1354 - val_acc: 0.9648
Epoch 7/15
60000/60000 [==============================] - 43s - loss: 0.0199 - acc: 0.9930 - val_loss: 0.1267 - val_acc: 0.9705
Epoch 8/15
60000/60000 [==============================

In [28]:

score = TCN.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.17482566874956265
Test accuracy: 0.9702
